In [17]:
from bs4 import BeautifulSoup as bs
import requests as re
import csv

# main url
url = 'https://www.uottawa.ca/en'


def getData():
    #for storing data
    data = []
    res = re.get(url)
    if res.status_code == 200:
        soup = bs(res.text,'html.parser')
        main_program_course = soup.find('div', class_='menu-childs__tabpanels')
        main_program_course_name = main_program_course.find('span',class_=["link--with-icon__content",  "link--with-icon__content--inline",  "headline", "headline--5"]).text
        main_program_course_link = main_program_course.findAll('a', class_=['link', 'link--dark-red', 'link--with-icon'])[2].get('href')
        # print(main_program_course_name, main_program_course_link)
    
    res_1 = re.get(main_program_course_link)
    if res_1.status_code == 200:
        soup_1 = bs(res_1.text, 'html.parser')
        program = soup_1.find('div', class_='two-column-link-list__wrapper').findAll('a', class_=['link', 'link--with-icon'])[2]
        program_name = program.find('span').text
        program_link = program.get('href')
        # print(program_name, program_link)
        res_new = re.get(program_link)
        if res_new.status_code == 200:
            s = bs(res_new.text, 'html.parser')
            all_sub_programs = s.find('section', class_=['links-cols-nav container']).findAll('a', class_='link')
            
            sub_programs_details = {}
            for sub_program in all_sub_programs:
                sub_programs_details[sub_program.find('span').text] = sub_program.get('href')
            for sub_program in sub_programs_details:
                res_2 = re.get(sub_programs_details[sub_program])
                if res_2.status_code == 200:
                    s_1 = bs(res_2.text, 'html.parser')

                    program_des = s_1.findAll('div', class_=['typo-bol-22', 'related-articles__item-text', 'el-full-height'])
                    program_duration = program_des[0].text
                    program_location = program_des[1].text
                    program_des_text = s_1.findAll('div', class_=['field', 'field--name-field-pp__intro', 'field--type-entity-reference-revisions', 'field--label-hidden', 'field__items'])[3].text.strip()
                    temp = [
                        url, 
                        main_program_course_name,
                        main_program_course_link,
                        program_name,
                        program_link,
                        sub_program,
                        sub_programs_details[sub_program],
                        program_duration,
                        program_location,
                        program_des_text
                    ]
                    data.append(temp)
    
    #header of data
    header = [
        'Main URL',
        'Main Program',
        'Main Program Link',
        'Under Graduation Program',
        'Under Graduation Porgram Link',
        'Course',
        'Course Link',
        'Course Duration',
        'Course Location',
        'Course Description'
    ]
    data.insert(0, header)
    #convertion into a csv file
    with open('scraped_data.csv', 'w') as f:
        csv_writer = csv.writer(f)
        # Write the CSV data to the file
        csv_writer.writerows(data)
    print('Data Saved in scraped_data.csv File !')

if __name__ == '__main__':
    getData()


Undergraduate programs https://www.uottawa.ca/study/this-is-uottawa/programs-suit-you
Engineering https://www.uottawa.ca/faculty-engineering/undergraduate-studies/programs
Data Saved in scraped_data.csv File !
